# Wrapping a template library

A template library is a library where there are only template classes that can be instantiated.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider various *C++* template classes instantiations during the `Parse` step.
It is therefore required to install the `clanglite` `parser`.

The **Standard Template Library (STL)** library is a *C++* library that provides a set of common *C++* template classes such as containers and associative arrays.
These classes can be used with any built-in or user-defined type that supports some elementary operations (e.g., copying, assignment).
It is divided in four components called algorithms, containers, functional and iterators.
**STL** containers (e.g., `std::vector`, `std::set`) are used in many *C++* libraries.
In such a case, it does not seem relevant that every wrapped *C++* library contains wrappers for usual **STL** containers (e.g., `std::vector< double >`, `std::set< int >`).
We therefore proposed *Python* bindings for some sequence containers (e.g., `vector` of the `std` namespace) and associative containers (e.g., `set`, `unordered_set` of the `std` namespace).
These template instantiations are done for various *C++* fundamental types (e.g., `int`, `unsigned long int`, `double`) and the `string` of the `std` namespace.
For ordered associative containers only the `std::less` comparator was used.
For the complete procedure refer to the `AutoWIG.py` file situed at the root of the **STL** [repository](https://github.com/StatisKit/STL).


We here aim at presenting how template libraries can be wrapped.
First, we need:

* to detect if the operating system (OS) is a Windows OS or a Unix OS.

In [1]:
import platform
is_windows = any(platform.win32_ver())

* to detect the version of *Python* installed and to save it in the `PYTHON_VERSION` environment variable.

In [2]:
import os
import sys
os.environ['PYTHON_VERSION'] = str(sys.version_info.major) + '.' + str(sys.version_info.minor)

In this notebook, we do not need to import **AutoWIG** since **SCons** is configured to use the **Boost.Python** tool installed with **AutoWIG** that can be used to generate wrappers (see the `../git/STL/src/cpp/SConscript` file).

In [3]:
if is_windows:
    !pygmentize ..\git\STL\src\cpp\SConscript
else:
    !pygmentize ../git/STL/src/cpp/SConscript

# -*-python-*-

VECTORS = ['Index',
           'int',
           'double',
           'std::string']

SETS = { 'less': ['Index',
                  'int',
                  'double',
                  'std::string'],
         'none': ['Index']}

HEADER = """\
#ifndef STATISKIT_STL_H
#define STATISKIT_STL_H

#include <vector>
#include <set>
#include <unordered_set>
#include <string>

#if defined WIN32 || defined _WIN32 || defined __CYGWIN__
    #ifdef LIBSTATISKIT_STL
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllexport))
        #else
            #define STATISKIT_STL_API __declspec(dllexport)
        #endif
    #else
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllimport))
        #else
            #define STATISKIT_STL_API __declspec(dllimport)
        #endif
    #endif
#else
    #if __GNUC__ >= 4
        #define STATISKIT_STL_API __attribute__ ((visibility ("default")))
    #else
        #define STATISKIT_STL_API
   

The controller is registered in the `../git/STL/src/cpp/AutoWIG.py` file

In [4]:
if is_windows:
    !pygmentize ..\git\STL\src\cpp\AutoWIG.py
else:
    !pygmentize ../git/STL/src/cpp/AutoWIG.py

import autowig
import itertools

def controller(asg, **kwargs):
    autowig.controller.plugin = 'default'
    asg = autowig.controller(asg, **kwargs)
    for function in asg['::statiskit::stl'].functions():
        if function.localname in ['generator', 'insert']:
            parameter = function.parameters[0].qualified_type.desugared_type
            if parameter.is_class:
                function.parent = parameter.unqualified_type
    for cls in asg['class ::std::vector'].specializations(partial = False):
        for method in cls.methods():
            if method.localname in ['resize', 'shrink_to_fit', 'operator[]']:
                if isinstance(method.boost_python_export, bool):
                    method.boost_python_export = False
        for constructor in cls.constructors():
            if not(constructor.nb_parameters == 0 or constructor.nb_parameters == 1 and constructor.parameters[0].qualified_type.unqualified_type == cls):
                if isinstance(constructor.boost_p

Then, in addition to the **STL** library, the **StatisKit.STL** library has to be installed in order to have access to some functionalities.
To do so, we use available **Conda** recipes.

In [5]:
!conda remove libstatiskit_stl -y
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\STL\bin\conda\libstatiskit_stl -c statiskit
else:
    !conda build --python=$PYTHON_VERSION ../git/STL/bin/conda/libstatiskit_stl -c statiskit
!conda install -y libstatiskit_stl --use-local -c statiskit

Attempting to finalize metadata for libstatiskit_stl
INFO:conda_build.metadata:Attempting to finalize metadata for libstatiskit_stl
BUILD START: [u'libstatiskit_stl-1.0.0-h179febd_0.tar.bz2']

The following NEW packages will be INSTALLED:

    ca-certificates: 2017.08.26-h1d4fec5_0           
    certifi:         2017.7.27.1-py27h9ceb091_0      
    libedit:         3.1-heed3624_0                  
    libffi:          3.2.1-h4deb6c0_3                
    libgcc-ng:       7.2.0-h7cc24e2_2                
    libstdcxx-ng:    7.2.0-h7a57d05_2                
    libtoolchain:    1.0.0-py27h6d5222f_0       local
    ncurses:         6.0-h06874d7_1                  
    openssl:         1.0.2l-h077ae2c_5               
    path.py:         10.3.1-py27hc258cac_0           
    pip:             9.0.1-py27ha730c48_4            
    python:          2.7.14-h89e7a4a_22              
    python-scons:    3.0.0-py27h8a4cdf6_0       local
    pyyaml:          3.12-py27h2d70dd7_1             
    

As presented below, in order to wrap a template library, the user needs to write headers containing aliases for desired template class instantiations (see the `../git/STL/src/cpp/STL.h` file).

In [6]:
if is_windows:
    !pygmentize ..\git\STL\src\cpp\STL.h
else:
    !pygmentize ../git/STL/src/cpp/STL.h

#ifndef STATISKIT_STL_H
#define STATISKIT_STL_H

#include <vector>
#include <set>
#include <unordered_set>
#include <string>

#if defined WIN32 || defined _WIN32 || defined __CYGWIN__
    #ifdef LIBSTATISKIT_STL
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllexport))
        #else
            #define STATISKIT_STL_API __declspec(dllexport)
        #endif
    #else
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllimport))
        #else
            #define STATISKIT_STL_API __declspec(dllimport)
        #endif
    #endif
#else
    #if __GNUC__ >= 4
        #define STATISKIT_STL_API __attribute__ ((visibility ("default")))
    #else
        #define STATISKIT_STL_API
    #endif
#endif

namespace statiskit
{
    typedef unsigned long int Index;
    typedef std::set< Index > Indices;

    namespace stl
    {

        template<class T>
            class Generator
            {
                public:
                    Generat

Once these preliminaries are done, we can proceed to the actual generation of wrappers for the **STL** library.
To do so, we need then to install the *C++* headers. 
This is done using the `cpp` target in **SCons**.

In [7]:
if is_windows:
    !scons cpp -C ..\git\STL
else:
    !scons cpp -C ../git/STL

scons: Entering directory `/home/pfernique/Desktop/develop/FP17/share/git/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/src/cpp/STL.h" as "/home/pfernique/.miniconda2/envs/FP17/include/statiskit/stl/STL.h"
g++ -o /home/pfernique/.miniconda2/envs/FP17/lib/libstatiskit_stl.so -shared build/src/cpp/STL.os -L/home/pfernique/.miniconda2/envs/FP17/lib
scons: done building targets.


Once the headers have been installed in the system, we parse headers with relevant compilation flags.
This is done using the `autowig` target in **SCons**.

In [8]:
if is_windows:
    !scons autowig -C ..\git\STL
else:
    !scons autowig -C ../git/STL

scons: Entering directory `/home/pfernique/Desktop/develop/FP17/share/git/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/src/cpp/AutoWIG.py" as "/home/pfernique/.miniconda2/envs/FP17/lib/python2.7/site-packages/scons_tools/site_autowig/controller/statiskit_stl.py"
autowig: Generating Boost.Python interface ...
/home/pfernique/.miniconda2/envs/FP17/lib/python2.7/site-packages/clanglite/autowig_parser.py:844: Warning: 
  warnings.warn('', Warning)
scons: done building targets.


Here is the list of the generated wrappers (untracked files).

In [9]:
if is_windows:
    !cd ..\git\STL & git status
else:
    !cd ../git/STL && git status

HEAD detached at 719c594
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	src/py/statiskit/stl/_stl.py
	src/py/wrapper/_stl.cpp
	src/py/wrapper/_stl.h
	src/py/wrapper/wrapper_107131f9768c56e794a9b0de728d1738.cpp
	src/py/wrapper/wrapper_10b14312eeb655268489cd34090870cf.cpp
	src/py/wrapper/wrapper_3b59a0980c80518c808634f7a84dc3cd.cpp
	src/py/wrapper/wrapper_448c20257e485acda59dc59305fceb58.cpp
	src/py/wrapper/wrapper_476c1c1f206251dba7af53c48f3f6e42.cpp
	src/py/wrapper/wrapper_6436891c9b6854f494789a812891cbe5.cpp
	src/py/wrapper/wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp
	src/py/wrapper/wrapper_745e4933f5b250d9bc5232fe864d0cf7.cpp
	src/py/wrapper/wrapper_74bc331707c1505eb2f807cae7c32bbb.cpp
	src/py/wrapper/wrapper_858bbf0613575e2ebe4b0e5902107ad6.cpp
	src/py/wrapper/wrapper_882a8dd1e64a51b4a9da29ab852a852e.cpp
	src/py/wrapper/wrapper_8f249c2863945cb59771b97c53bdafec.cpp
	src/py/wrapper/wrapper_c047f2c3135554ceb57f166fd404cfc8.cpp
	src/py/wrapper/wrapp

And here, we present the wrappers generated for the `std::vector< int >` class.

In [10]:
if is_windows:
    !pygmentize ..\git\STL\src\py\wrapper\wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp
else:
    !pygmentize ../git/STL/src/py/wrapper/wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp

#include "_stl.h"



namespace autowig
{

    void method_decorator_bb1e0852f2ca56c094260a03787426c7(class ::std::vector< int, class ::std::allocator< int > > & instance, ::std::vector< int, class ::std::allocator< int > >::size_type  param_in_0, int param_out) { instance.at(param_in_0) = param_out; }
    void method_decorator_7ec1ac72b0b05f3a9707175bcd5da0bd(class ::std::vector< int, class ::std::allocator< int > > & instance, int param_out) { instance.front() = param_out; }
    void method_decorator_ed1cf37568ed54cbbd326e6ccbe5f27d(class ::std::vector< int, class ::std::allocator< int > > & instance, int param_out) { instance.back() = param_out; }
}

#if defined(_MSC_VER)
    #if (_MSC_VER == 1900)
namespace boost
{
    template <> class ::std::vector< int, class ::std::allocator< int > > const volatile * get_pointer<class ::std::vector< int, class ::std::allocator< int > > const volatile >(class ::std::vector< int, class ::std::allocator< int > > const volatile *c) { return c; }
}
 

    class_6b9ae5eac40858c9a0f5e6e21c15d1d3.def("push_back", method_pointer_341df5e2719858f39df095cac9121eaf, "");
    class_6b9ae5eac40858c9a0f5e6e21c15d1d3.def("pop_back", method_pointer_bbaecaa6c9535f04a1ffda1223792c23, "");
    class_6b9ae5eac40858c9a0f5e6e21c15d1d3.def("swap", method_pointer_433012ce9fb655529590bcbd5d85150b, "");
    class_6b9ae5eac40858c9a0f5e6e21c15d1d3.def("clear", method_pointer_201a5d8f6cc15fd1b83d45af764f3905, "");

    struct vector_6b9ae5eac40858c9a0f5e6e21c15d1d3_from_python
    {
        vector_6b9ae5eac40858c9a0f5e6e21c15d1d3_from_python()
        {
            boost::python::converter::registry::push_back(
                &convertible,
                &construct,
                boost::python::type_id< class ::std::vector< int, class ::std::allocator< int > > >());
        }

        static void* convertible(PyObject* obj_ptr)
        { return obj_ptr; }

        static void construct(PyObject* obj_ptr, boost::python::converter::rvalue_from_python_stage

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.
To do so, we use available **Conda** recipes.

In [11]:
!conda remove python-statiskit_stl -y
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\STL\bin\conda\python-statiskit_stl -c statiskit -c conda-forge
else:
    !conda build --python=$PYTHON_VERSION ../git/STL/bin/conda/python-statiskit_stl -c statiskit -c conda-forge
!conda install -y python-statiskit_stl --use-local -c statiskit -c conda-forge --force

Attempting to finalize metadata for python-statiskit_stl
INFO:conda_build.metadata:Attempting to finalize metadata for python-statiskit_stl
BUILD START: [u'python-statiskit_stl-3.3.1-py27h6d1d95f_0.tar.bz2']

The following NEW packages will be INSTALLED:

    alabaster:                0.7.10-py27_1         conda-forge
    asn1crypto:               0.22.0-py27_0         conda-forge
    babel:                    2.5.1-py27_0          conda-forge
    bzip2:                    1.0.6-1               conda-forge
    ca-certificates:          2017.7.27.1-0         conda-forge
    certifi:                  2017.7.27.1-py27_0    conda-forge
    cffi:                     1.10.0-py27_0         conda-forge
    chardet:                  3.0.4-py27_0          conda-forge
    coverage:                 4.4.1-py27_0          conda-forge
    cryptography:             2.0.3-py27_0          conda-forge
    docutils:                 0.14-py27_0           conda-forge
    enum34:                   1.1.6-py27

In file included from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/detail/is_xxx.hpp:8:0,
                 from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/detail/is_auto_ptr.hpp:9,
                 from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/detail/copy_ctor_mutates_rhs.hpp:8,
                 f

In file included from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/converter/arg_to_python.hpp:10:0,
                 from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/call.hpp:15,
                 from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/object_core.hpp:14,
                 from /home/pferniqu

In file included from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python.hpp:41:0,
                 from build/src/py/wrapper/_stl.h:4:
/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/make_constructor.hpp: In member function 'void boost::python::detail::install_holder<T>::dispatch(U*, mpl_::true_) const':
/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehol

g++ -o build/src/py/wrapper/wrapper_8f249c2863945cb59771b97c53bdafec.os -c -x c++ -std=c++11 -fvisibility=hidden -fmax-errors=0 -Wl,--no-undefined -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -I/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include -I/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/python2.7 build/src/py/wrapper/wrapper_8f249c2863945cb59771b97c53bdafec.cpp
g++ -o build/src/py/wrapper/wrapper_dc5522842bc75d8b9ac7b46020c60854.os -c -x c++ -std=c++11 -fvisibility=hidden -fmax-errors=0 -Wl,--no-undefined -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_N

...

In file included from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/object/function_handle.hpp:10:0,
                 from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/converter/arg_to_python.hpp:19,
                 from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/call.hpp:15,
                 from /ho

In file included from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/detail/is_xxx.hpp:8:0,
                 from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/detail/is_auto_ptr.hpp:9,
                 from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/detail/copy_ctor_mutates_rhs.hpp:8,
                 f

In file included from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python.hpp:41:0,
                 from build/src/py/wrapper/_stl.h:4,
                 from build/src/py/wrapper/wrapper_882a8dd1e64a51b4a9da29ab852a852e.cpp:1:
/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/make_constructor.hpp: In member function 'void boost::python::detail::install_holder<T>::dispatch(U*, mpl_::true_) const':
/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placeho

In file included from /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python.hpp:41:0,
                 from build/src/py/wrapper/_stl.h:4,
                 from build/src/py/wrapper/_stl.cpp:1:
/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/include/boost/python/make_constructor.hpp: In member function 'void boost::python::detail::install_holder<T>::dispatch(U*, mpl_::true_) const':
/home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla

Packaging python-statiskit_stl-3.3.1-py27h6d1d95f_0
INFO:conda_build.build:Packaging python-statiskit_stl-3.3.1-py27h6d1d95f_0
compiling .pyc files...
number of files: 13
Fixing permissions
patchelf: file: /home/pfernique/.miniconda2/conda-bld/python-statiskit_stl_1509613880135/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place/lib/python2.7/site-packages/statiskit/stl/__stl.so
    setting rpath to: $ORIGIN/../../../..
Fixing permissions
Detected hard-coded path in binary file lib/python2.7/site-packages/statiskit/stl/__stl.so
updating: python-statiskit_stl-3.3.1-py27h6d1d95f_0.tar.bz2
Updating index at /home/pfernique/.miniconda2/conda-bld/linux-64 to make package installable with dependencies
INFO:conda_build.build:Updating index at /home/pfernique/.miniconda2/conda-bld/linux-64 to make package installable with dependencies
Updating index at /home/pferni

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [12]:
from statiskit.stl import VectorInt
v = VectorInt()
v.push_back(-1)
v.push_back(0)
v.push_back(1)
v

(-1, 0, 1)

In [13]:
list(v)

[-1, 0, 1]

In [14]:
v[0]

-1

In [15]:
v[0] = -2
v[0]

-2

In [16]:
VectorInt([0, 1])

(0, 1)

Here is a report concerning objects wrapped using this notebook.

In [17]:
import fp17
import os
import pickle
with open(os.path.join(os.environ['SCONSFLAGS'].lstrip('--site-dir='),
                       'site_autowig',
                       'ASG',
                       'statiskit_stl.pkl'), 'rb') as filehandler:
    asg = pickle.load(filehandler)
fp17.report(asg)

Headers: 40 (34241 SLOC)
Enumerations: 1 (0 wrapped)
Variables: 34 (1 wrapped)
Functions: 675 (133 wrapped)
Classes: 112 (17 wrapped)
